<a href="https://colab.research.google.com/github/pravincoder/binance_trade_analysis/blob/main/raw_csv_to_trade_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
url = "https://github.com/pravincoder/binance_trade_analysis/raw/refs/heads/main/TRADES_CopyTr_90D_ROI.csv"
df = pd.read_csv(url)



In [20]:
# Lets focus on the trade_hist data in each row
df.Trade_History.head(5)

,Trade_History
0,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."


In [21]:
import ast
print(df.info())
print(df.head())

# Check if Trade_History column contains valid data
def parse_trade_history(history):
    try:
        return ast.literal_eval(history) if isinstance(history, str) else []
    except Exception as e:
        return []

# Apply parsing function
df["Trade_History"] = df["Trade_History"].apply(parse_trade_history)

# Display parsed data
print("Sample Trade History:")
print(df["Trade_History"].head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB
None
              Port_IDs                                      Trade_History
0  3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1  4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2  3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3  3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4  3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...
Sample Trade History:
0    [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1    [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2    [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3    [{'time': 1718678214000, 'symbol'

In [22]:
# Extract trade details into separate rows
trade_data = []
for idx, row in df.iterrows():
    port_id = row["Port_IDs"]
    trade_history = row["Trade_History"]

    for trade in trade_history:
        trade_data.append({
            "Port_IDs": port_id,
            "time": trade.get("time", None),
            "symbol": trade.get("symbol", None),
            "price": trade.get("price", None),
            "quantity": trade.get("quantity", None),
            "trade_type": trade.get("side", None),  # Assuming 'side' is 'BUY' or 'SELL'
            "Profit/Loss": trade.get("realizedProfit", None)
        })

# Convert extracted data into a DataFrame
df_trades = pd.DataFrame(trade_data)

# Convert timestamp to readable date
df_trades["time"] = pd.to_datetime(df_trades["time"], unit="ms")

# Display transformed data
print("Processed Trade Data:")
print(df_trades.head(20))



Processed Trade Data:
               Port_IDs                time    symbol      price    quantity  \
0   3925368433214965504 2024-06-20 16:07:36   SOLUSDT  132.53700  1988.05500   
1   3925368433214965504 2024-06-20 16:06:58  DOGEUSDT    0.12182  1398.98088   
2   3925368433214965504 2024-06-20 16:06:58  DOGEUSDT    0.12182   197.47022   
3   3925368433214965504 2024-06-20 16:06:56  DOGEUSDT    0.12182    16.56752   
4   3925368433214965504 2024-06-20 16:06:56  DOGEUSDT    0.12182    92.21774   
5   3925368433214965504 2024-06-20 16:06:56  DOGEUSDT    0.12182   251.43648   
6   3925368433214965504 2024-06-20 16:06:56  DOGEUSDT    0.12182    43.61156   
7   3925368433214965504 2024-06-20 16:06:47   SOLUSDT  132.58500   795.51000   
8   3925368433214965504 2024-06-20 16:06:47   SOLUSDT  132.58500  1193.26500   
9   3925368433214965504 2024-06-20 13:21:24   FILUSDT    4.45400   697.05100   
10  3925368433214965504 2024-06-20 13:21:24   FILUSDT    4.45400     6.23560   
11  39253684332149

In [ ]:
# Save to CSV for further analysis
df_trades.to_csv("trades.csv", index=False)